In [ ]:
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
import pandas as pd
import shutil
from tqdm import tqdm

In [ ]:
f1=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/Hereford_raw_data/Hereford_protein.tsv", delimiter='\t')

In [ ]:
f1.columns

Index(['Accession', 'Begin', 'End', 'Chromosome', 'Orientation', 'Name',
       'Symbol', 'Gene ID', 'Gene Type', 'Transcripts accession',
       'Protein accession', 'Protein length', 'Locus tag'],
      dtype='object')

In [ ]:
unique_chr=f1["Chromosome"].unique()
unique_chr

array(['MT', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11',
       '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22',
       '23', '24', '25', '26', '27', '28', '29', 'X', 'Y', nan],
      dtype=object)

In [ ]:
for chr in unique_chr:
  # extract the data as chromosome wise
  chr_df=f1[f1["Chromosome"]==chr]

  # save the out file of hereford(HF) based on chromosome
  out_file=f"HF{chr}.txt"

  # save the DataFrame as a seperate file
  chr_df.to_csv(out_file, sep='\t', index=False)

  # save the out file into drive
  shutil.copy(out_file, "/content/drive/MyDrive/Hereford_VarAnalysis/Hereford_Protein" )

In [ ]:
qtl_df=pd.read_csv('/content/drive/MyDrive/QTL_analysis_051223/QTL_ARS_UCD1_2/qtl_data_Chr.1.txt', delimiter='\t', encoding='latin1')

In [ ]:
qtl_df.shape

(3579, 6)

In [ ]:
qtl_df.head()

,Chromosome,Source,QTL Class,QTL Span1,QTL Span2,Description
0,Chr.2,Animal QTLdb,Meat_and_Carcass_Association,90562,90566,QTL_ID=225243;Name=Shear force;Abbrev=;PUBMED_...
1,Chr.2,Animal QTLdb,Reproduction_Association,292762,292766,QTL_ID=39029;Name=Pregnancy rate;Abbrev=;PUBME...
2,Chr.2,Animal QTLdb,Reproduction_Association,292762,292766,QTL_ID=39030;Name=Stillbirth;Abbrev=;PUBMED_ID...
3,Chr.2,Animal QTLdb,Production_Association,292762,292766,QTL_ID=39031;Name=PTA type;Abbrev=;PUBMED_ID=2...
4,Chr.2,Animal QTLdb,Exterior_Association,292762,292766,QTL_ID=39032;Name=Udder attachment;Abbrev=;PUB...


In [ ]:
hf_df=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/Hereford_Protein/HF1.txt", delimiter="\t")

In [ ]:
hf_df.shape

(1643, 13)

In [ ]:
# Add column as start of next gene to find variation in intergenic region
hf_df["start_of_next_gene"] = hf_df["Begin"].shift(-1).fillna(0)

In [ ]:
hf_df.shape

(1566, 14)

In [ ]:
hf_df.head()

,Accession,Begin,End,Chromosome,Orientation,Name,Symbol,Gene ID,Gene Type,Transcripts accession,Protein accession,Protein length,Locus tag
0,NC_037328.1,207933,217580,1,minus,ankyrin repeat domain-containing protein 26-like,LOC112447009,112447009,pseudogene,NaN,NaN,NaN,NaN
1,NC_037328.1,228440,283123,1,minus,uncharacterized LOC101903639,LOC101903639,101903639,lncRNA,XR_003035135.2,NaN,NaN,NaN
2,NC_037328.1,284711,290531,1,minus,uncharacterized LOC112447074,LOC112447074,112447074,lncRNA,XR_003035148.2,NaN,NaN,NaN
3,NC_037328.1,328577,347809,1,minus,uncharacterized LOC112447010,LOC112447010,112447010,protein-coding,XM_059890683.1,XP_059746666.1,375.0,NaN
4,NC_037328.1,419848,423189,1,minus,uncharacterized LOC100138661,LOC100138661,100138661,lncRNA,XR_003035143.2,NaN,NaN,NaN


In [ ]:
Chr_list=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, "X"]

In [ ]:
def hf_qtl(qtl_df, hf_df, chr, out_loc):
  final_df=pd.DataFrame(columns=list(hf_df)+["qtlgene_type"]+["QTL Class"]+["QTL Span1"]+["QTL Span2"]+["Description"])
  # Finding the presence of QTL in Hereford annoted protein
  for i, row in tqdm(qtl_df.iterrows()):
    # Check for the at Begin
    if hf_df[(hf_df["Begin"]==row["QTL Span1"]) & (hf_df["End"]>row["QTL Span2"])].shape[0]>=1:
        tmp_df = hf_df[(hf_df["Begin"]==row["QTL Span1"]) & (hf_df["End"]>row["QTL Span2"])].copy(deep=True)
        tmp_df["qtlgene_type"] = "at_start"
        tmp_df["QTL Class"] = row["QTL Class"]
        tmp_df["QTL Span1"] = row["QTL Span1"]
        tmp_df["QTL Span2"] = row["QTL Span2"]
        tmp_df["Description"] = row["Description"]
        final_df = pd.concat([final_df,tmp_df]).reset_index(drop=True)
    # check for the at end
    elif hf_df[(hf_df["Begin"]<row["QTL Span1"]) & (hf_df["End"]==row["QTL Span2"])].shape[0]>=1:
        tmp_df = hf_df[(hf_df["Begin"]<row["QTL Span1"]) & (hf_df["End"]==row["QTL Span2"])].copy(deep=True)
        tmp_df["qtlgene_type"] = "at_end"
        tmp_df["QTL Class"] = row["QTL Class"]
        tmp_df["QTL Span1"] = row["QTL Span1"]
        tmp_df["QTL Span2"] = row["QTL Span2"]
        tmp_df["Description"] = row["Description"]
        final_df = pd.concat([final_df,tmp_df]).reset_index(drop=True)
    # Check for the within gene
    elif hf_df[(hf_df["Begin"]<row["QTL Span1"]) & (hf_df["End"]>row["QTL Span2"])].shape[0]>=1:
        tmp_df = hf_df[(hf_df["Begin"]<row["QTL Span1"]) & (hf_df["End"]>row["QTL Span2"])].copy(deep=True)
        tmp_df["qtlgene_type"] = "within_gene"
        tmp_df["QTL Class"] = row["QTL Class"]
        tmp_df["QTL Span1"] = row["QTL Span1"]
        tmp_df["QTL Span2"] = row["QTL Span2"]
        tmp_df["Description"] = row["Description"]
        final_df = pd.concat([final_df,tmp_df]).reset_index(drop=True)
    # Check for the near Begin gene with integenic
    elif hf_df[(hf_df["Begin"]>row["QTL Span1"]) & (hf_df["Begin"]<row["QTL Span2"])].shape[0]>=1:
        tmp_df = hf_df[(hf_df["Begin"]>row["QTL Span1"]) & (hf_df["Begin"]<row["QTL Span2"])].copy(deep=True)
        tmp_df["qtlgene_type"] = "near_start_gene with intergenic"
        tmp_df["QTL Class"] = row["QTL Class"]
        tmp_df["QTL Span1"] = row["QTL Span1"]
        tmp_df["QTL Span2"] = row["QTL Span2"]
        tmp_df["Description"] = row["Description"]
        final_df = pd.concat([final_df,tmp_df]).reset_index(drop=True)
    # Check for the near End gene with intergenic
    elif hf_df[(hf_df["End"]>row["QTL Span1"]) & (hf_df["End"]<row["QTL Span2"])].shape[0]>=1:
        tmp_df = hf_df[(hf_df["End"]>row["QTL Span1"]) & (hf_df["End"]<row["QTL Span2"])].copy(deep=True)
        tmp_df["qtlgene_type"] = "near_stop_gene with intergenic"
        tmp_df["QTL Class"] = row["QTL Class"]
        tmp_df["QTL Span1"] = row["QTL Span1"]
        tmp_df["QTL Span2"] = row["QTL Span2"]
        tmp_df["Description"] = row["Description"]
        final_df = pd.concat([final_df,tmp_df]).reset_index(drop=True)
    # Check for the at Begin with intergenic gene
    elif hf_df[(hf_df["Begin"]>row["QTL Span1"]) & (hf_df["Begin"]==row["QTL Span2"])].shape[0]>=1:
        tmp_df = hf_df[(hf_df["Begin"]>row["QTL Span1"]) & (hf_df["Begin"]==row["QTL Span2"])].copy(deep=True)
        tmp_df["qtlgene_type"] = "at_start with intergenic_gene"
        tmp_df["QTL Class"] = row["QTL Class"]
        tmp_df["QTL Span1"] = row["QTL Span1"]
        tmp_df["QTL Span2"] = row["QTL Span2"]
        tmp_df["Description"] = row["Description"]
        final_df = pd.concat([final_df,tmp_df]).reset_index(drop=True)
    # check for the at end with intergenic
    elif hf_df[(hf_df["End"]==row["QTL Span1"]) & (hf_df["End"]<row["QTL Span2"])].shape[0]>=1:
        tmp_df = hf_df[(hf_df["End"]==row["QTL Span1"]) & (hf_df["End"]<row["QTL Span2"])].copy(deep=True)
        tmp_df["qtlgene_type"] = "at_end with intergenic_gene"
        tmp_df["QTL Class"] = row["QTL Class"]
        tmp_df["QTL Span1"] = row["QTL Span1"]
        tmp_df["QTL Span2"] = row["QTL Span2"]
        tmp_df["Description"] = row["Description"]
        final_df = pd.concat([final_df,tmp_df]).reset_index(drop=True)
    # Check for the intergenic gene
    elif hf_df[(hf_df["End"]<row["QTL Span1"]) & (hf_df["hfstart_of_next_gene"]>row["QTL Span2"])].shape[0]>=1:
        tmp_df = hf_df[(hf_df["End"]<row["QTL Span1"]) & (hf_df["hfstart_of_next_gene"]>row["QTL Span2"])].copy(deep=True)
        tmp_df["qtlgene_type"] = "intergenic_gene"
        tmp_df["QTL Class"] = row["QTL Class"]
        tmp_df["QTL Span1"] = row["QTL Span1"]
        tmp_df["QTL Span2"] = row["QTL Span2"]
        tmp_df["Description"] = row["Description"]
        final_df = pd.concat([final_df,tmp_df]).reset_index(drop=True)
    else:
        pass

  # Generate output file
  print("hfqtl", final_df.shape)
  out_file=f"hf_qtl{chr}.csv"
  final_df.to_csv(out_file)
  shutil.copy(out_file, out_loc)

  # remove duplicates
  hfqtl_wd = final_df.drop_duplicates(subset=['Symbol', 'QTL Class', 'QTL Span1', "QTL Span2"])
  print("hfqtl_wd", hfqtl_wd.shape)
  out_file1=f"hf_qtl_wd{chr}.csv"
  hfqtl_wd.to_csv(out_file1)
  shutil.copy(out_file1, out_loc)

  # find variation not present in gene and intergene region called as other non coding region
  final_df1 = pd.DataFrame(columns=hf_df.columns)
  for index, row in tqdm(hf_df.iterrows()):
    # Check if 'startpos.q' is present in any row of file2
    if row['Begin'] not in final_df['Begin'].values:
        # Append the row to the DataFrame if not found
        final_df1 = final_df1.append(row, ignore_index=True)

  # Save the DataFrame with rows not found in file2 to a new CSV file
  out_file2=f"hfnon_qtl{chr}.csv"
  final_df1["QTL Class"]="Nil"
  final_df1.to_csv(out_file2, index=False)
  shutil.copy(out_file2, out_loc)
  print("hf_nonqtl", final_df1.shape)

  # Merge the data has QTL and non QTL
  HF_QTL=pd.DataFrame(columns=list(hfqtl_wd))
  HF_QTL=pd.concat([HF_QTL, hfqtl_wd, final_df1])
  print("HF_QTL", HF_QTL.shape)
  out_file3=f"HF_QTL{chr}.csv"
  HF_QTL.to_csv(out_file3, index=False)
  shutil.copy(out_file3, out_loc)


In [ ]:
Chr_list=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, "X"]

In [ ]:
for chr in Chr_list:
  in_file1=f"/content/drive/MyDrive/QTL_analysis_051223/QTL_ARS_UCD1_2/qtl_data_Chr.{chr}.txt"
  in_file2=f"/content/drive/MyDrive/Hereford_VarAnalysis/Hereford_Protein/HF{chr}.txt"
  out_loc=f"/content/drive/MyDrive/Hereford_VarAnalysis/Hereford_qtl/{chr}"
  qtl_df=pd.read_csv(in_file1, delimiter="\t", encoding='latin1')
  hf_df=pd.read_csv(in_file2, delimiter="\t")
  hf_df["hfstart_of_next_gene"] = hf_df["Begin"].shift(-1).fillna(0)
  print("hf_df", hf_df.shape)
  hf_qtl(qtl_df, hf_df, chr, out_loc)

hf_df (1643, 14)


3579it [00:39, 90.26it/s]


hfqtl (3657, 19)
hfqtl_wd (2441, 19)


0it [00:00, ?it/s]<ipython-input-64-8ad4a36f5561>:99: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df1 = final_df1.append(row, ignore_index=True)
1643it [00:03, 487.40it/s]


hf_nonqtl (994, 15)
HF_QTL (3435, 19)
hf_df (1566, 14)


...

hf_nonqtl (617, 15)
HF_QTL (3908, 19)
hf_df (1783, 14)


19550it [06:23, 51.00it/s]


hfqtl (19877, 19)
hfqtl_wd (8981, 19)


0it [00:00, ?it/s]<ipython-input-64-8ad4a36f5561>:99: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df1 = final_df1.append(row, ignore_index=True)
1783it [00:04, 421.28it/s]


hf_nonqtl (899, 15)
HF_QTL (9880, 19)


In [ ]:
f1=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/Hereford_qtl/hf_qtl_wd1.csv")
f2=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/Hereford_qtl/hfnon_qtl1.csv")

In [ ]:
f1.shape

(2441, 20)

In [ ]:
f2["QTL Class"]= "Nil"

In [ ]:
2441+994

3435

In [ ]:
f2.shape

(994, 15)

In [ ]:
hfqtl=pd.DataFrame(columns=list(f1))

In [ ]:
hfqtl=pd.concat([hfqtl, f1, f2])

In [ ]:
hfqtl.shape

(3435, 20)

In [ ]:
hfqtl.to_csv("HFQTL1.csv")
shutil.copy("HFQTL1.csv", "/content/drive/MyDrive/Hereford_VarAnalysis/Hereford_qtl")

'/content/drive/MyDrive/Hereford_VarAnalysis/Hereford_qtl/HFQTL1.csv'